# 타행의 이벤트를 알려주는 텔레그램 봇

## 알고리즘
## 1. 미리 만들어 둔 텔레그램 id를 파이썬에 연결합니다.
## 2. 기업,국민,농협,우리,하나,부산 은행의 이벤트 사이트의 페이지 소스중에 가장 최신의 이벤트 명을 받아서 txt에 저장합니다.
## 3. 만약 가장 최신의 이벤트명과 저장되어져있던 txt파일안의 이벤트 명이 다르고 이벤트 출시일이 당일이라면 크롬창으로 해당 은행 이벤트 창으로 접속합니다.
## 4. 접속이 된후 가장최신의 이벤트 사이트배너를 클릭한뒤 이벤트 명과 이벤트사진을 텔레그램봇을 통해서 전송합니다.

## ※작동을 원하시면 크롬드라이버를 c드라이브에 넣어두셔야 합니다!


In [9]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import urllib
import re
import os
import time
import datetime
import telegram

#현재 날짜를 받아오늘 함수
now = datetime.datetime.now()
nowdate = now.strftime('%Y.%m.%d')

#나의 telegram 토큰의 접속
my_token = '864879159:AAEtqMiXc9T1DlSMyhNuUC2LVzvA6nh5vkM'
bot = telegram.Bot(token = my_token) #봇을 생성합니다.

# 각은행의 사이트와 이벤트 관련된 부분만 슬라이스 할 변수입니다.
IBKurl = 'https://www.ibk.co.kr/event/ingListEvent.ibk?pageId=CM01060100&evnt_dscd=H'
IBK_sel = 'div[class="event_info"]'

KBurl = 'https://omoney.kbstar.com/quics?page=C016559#loading'
KB_sel = 'td > a'

NHurl = 'https://banking.nonghyup.com/servlet/IPNL5611R.view'
NH_sel = 'h2[class="tit"]'

WOORIurl = 'https://spot.wooribank.com/pot/Dream?withyou=EVEVT0001'
WOORI_sel = 'span[class="hidden"]'

HANAurl = 'https://www.kebhana.com/cont/news/news02/index.jsp?_menuNo=98781'
HANA_sel = 'p[class="ethumb"]'

BNKurl = 'https://www.busanbank.co.kr/ib20/mnu/BHPBKI393002005'
BNK_sel ='span[class="txt"]'


#크롬창을 엽니다.  밑에 보이는 headless는 크롬창을 띄우지않은 상태에서 진행하게 해주는 옵션입니다.
options = webdriver.ChromeOptions()
# options.add_argument('headless')
# options.add_argument('window-size=1920x1080')
# options.add_argument("disable-gpu")

driver = webdriver.Chrome('c:\chromedriver.exe', chrome_options=options)


#한국어만 추출하는 함수 입니다.
def Korean(sentence): #한글 추출 함수
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+') # 한글과 띄어쓰기를 제외한 모든 글자
    result = hangul.sub('',sentence) # 한글과 띄어쓰기를 제외한 모든 부분을 제거
    return result.strip()


def Html_print(url):  # 이벤트창의 페이지소스를 받는 함수
    req = requests.get(url)
    req.encoding = 'utf-8'
    html = req.text
    re_soup = BeautifulSoup(html, 'html.parser')
    return re_soup

def html_sel(soup,select): # 이벤트관련 소스를 받는 함수
    posts= soup.select(select)
    return str(posts)

def IBK_event_road(soup,select): # ibk의 이벤트명을 받는 함수
    name1 = html_sel(soup,IBK_sel)
    name2 = Korean(name1)
    name3 = name2.split('기간')
    return name3[0].strip()


def KB_event_road(soup,select):  # NH의 이벤트명을 받는 함수
    name = html_sel(soup, select).split('</a>')
    return Korean(name[0])
    
def NH_event_road(soup,select):  # NH의 이벤트명을 받는 함수
    name = html_sel(soup, NH_sel).split('\n')
    return Korean(name[1]).strip()

def WOORI_event_road(soup,select):  # NH의 이벤트명을 받는 함수
    name = html_sel(soup, select).split(',')
    name2 = name[5].strip()
    return Korean(name2)

def HANA_event_road(soup,select):  # NH의 이벤트명을 받는 함수
    name = html_sel(soup, select).split(',')
    return Korean(name[0])

def BNK_event_road(soup,select):  # NH의 이벤트명을 받는 함수
    name = html_sel(soup, select).split('</span>')
    return Korean(name[0])

#새로운 이벤트가 발생 했다면 IBK의 이벤트창으로 접속후 이벤트명과 이벤트사진 정보를 크롤링하고 텔레그램으로 메세지를 전달하는 함수
def IBK_png():
    driver.get(IBKurl)
    a=driver.find_element_by_xpath('//*[@id="contents_in"]/ul/li[1]/div/ul/li').text
    term = a[2:12]
    if(term== nowdate):
        driver.find_element_by_xpath('//*[@id="contents_in"]/ul/li[1]/div/a').click()
        time.sleep(1)
        img = driver.find_element_by_xpath('//*[@id="contents_in"]/table/tbody[1]/tr[3]/td/div/img[1]')
        src = img.get_attribute('src')
        title = driver.find_element_by_xpath('//*[@id="contents_in"]/table/tbody[1]/tr[1]/td').text
        bot.sendMessage(chat_id='@yyfactory', text ='IBK event - '+title)
        try:
            bot.sendPhoto(chat_id='@yyfactory', photo =src, timeout=20) 
            time.sleep(2)
        except Exception as e : pass
        bot.sendMessage(chat_id='@yyfactory', text = IBKurl)    
        
    else:
        None
    
#새로운 이벤트가 발생 했다면 KB의 이벤트창으로 접속후 이벤트명과 이벤트사진 정보를 크롤링하고 텔레그램으로 메세지를 전달하는 함수
def KB_png():
    driver.get(KBurl)
    a=driver.find_element_by_xpath('//*[@id="b033091"]/table/tbody/tr[2]/td[3]').text
    term = a[0:10]
    if(term!=nowdate):
        driver.find_element_by_xpath('//*[@id="b033091"]/table/tbody/tr[2]/td[2]/a').click()
        time.sleep(1)
        img = driver.find_element_by_xpath('//*[@id="b032977"]/div[2]/dl/dd/div[2]/div/div[1]/img')
        src = img.get_attribute('src')
        title = driver.find_element_by_xpath('//*[@id="b032977"]/div[2]/dl/dt').text
        bot.sendMessage(chat_id='@yyfactory', text ='KB event -'+title)
        try:
            bot.sendPhoto(chat_id='@yyfactory', photo =src, timeout=20) 
            time.sleep(2)
        except Exception as e : pass
        bot.sendMessage(chat_id='@yyfactory', text = KBurl)
        
        
    else: None
    
#새로운 이벤트가 발생 했다면 NH의 이벤트창으로 접속후 이벤트명과 이벤트사진 정보를 크롤링하고 텔레그램으로 메세지를 전달하는 함수
def NH_png():
    driver.get(NHurl)
    driver.find_element_by_xpath('//*[@id="IPZZ05040401"]/a').click()
    time.sleep(2)
    a=driver.find_element_by_class_name('write').text
    term= a[0:10].replace('/','.')
    
    if(term==nowdate):
        driver.find_element_by_xpath('//*[@id="form01"]/div/div/div[2]/div[2]/div[1]/div[1]/h2/a').click()
        time.sleep(1)
        try: 
            img = driver.find_element_by_xpath('//*[@id="event"]/div/p/img')
            src = img.get_attribute('src')
        except Exception as e : pass
        
        title = driver.find_element_by_class_name('board_tit').text
        bot.sendMessage(chat_id='@yyfactory', text ='NH event -'+title)
        try:
            bot.sendPhoto(chat_id='@yyfactory', photo =src, timeout=20) 
            time.sleep(2)
        except Exception as e : pass
        bot.sendMessage(chat_id='@yyfactory', text = NHurl)
        
        
    else: None

        
#새로운 이벤트가 발생 했다면 WOORI의 이벤트창으로 접속후 이벤트명과 이벤트사진 정보를 크롤링하고 텔레그램으로 메세지를 전달하는 함수
def WOORI_png():    
    driver.get(WOORIurl)
    a=driver.find_element_by_xpath('//*[@id="resultArea"]/div/dl[1]/dd/ul/li[1]/dl/dd[2]').text
    term = a[8:18]
    if(term==nowdate):
        driver.find_element_by_xpath('//*[@id="resultArea"]/div/dl[2]/dt/a/span/img').click()
        time.sleep(1)
        img = driver.find_element_by_xpath('//*[@id="content"]/div[1]/div[2]/div/img[1]')
        src = img.get_attribute('src')
        title = driver.find_element_by_xpath('//*[@id="content"]/div[1]/div[1]/h3').text
        bot.sendMessage(chat_id='@yyfactory', text ='WOORI event -'+ title)
        try:
            bot.sendPhoto(chat_id='@yyfactory', photo =src, timeout=20) 
            time.sleep(2)
        except Exception as e : pass
        bot.sendMessage(chat_id='@yyfactory', text = WOORIurl)
        
        
    else: None

        
#새로운 이벤트가 발생 했다면 HANA의 이벤트창으로 접속후 이벤트명과 이벤트사진 정보를 크롤링하고 텔레그램으로 메세지를 전달하는 함수        
def HANA_png():
    driver.get(HANAurl)
    a=driver.find_element_by_class_name('date').text
    term = a[0:10]
    if(term==nowdate):
        driver.find_element_by_xpath('//*[@id="contents"]/ul/li[1]/p/img').click()
        time.sleep(1)
        img = driver.find_element_by_xpath('//*[@id="contents"]/div[3]/table/tbody/tr[2]/td/div[2]/img')
        src = img.get_attribute('src')
        title = driver.find_element_by_xpath('//*[@id="contents"]/div[3]/table/tbody/tr[1]/th/h4').text
        bot.sendMessage(chat_id='@yyfactory', text ='HANA event -'+title)
        try:
            bot.sendPhoto(chat_id='@yyfactory', photo =src, timeout=20) 
            time.sleep(2)
        except Exception as e : pass
        bot.sendMessage(chat_id='@yyfactory', text = HANAurl)
        
       
    else: None

        
        
#새로운 이벤트가 발생 했다면 BNK의 이벤트창으로 접속후 이벤트명과 이벤트사진 정보를 크롤링하고 텔레그램으로 메세지를 전달하는 함수
def BNK_png():
    driver.get(BNKurl)
    a=driver.find_element_by_xpath('//*[@id="contentForm"]/div/ul[1]/li[1]/div/span[2]').text
    term = a[8:18]
    if(term==nowdate):
        driver.find_element_by_xpath('//*[@id="contentForm"]/div/ul[1]/li[1]/span/a/img').click()
        time.sleep(1)
        img = driver.find_element_by_xpath('//*[@id="con"]/div[2]/div[3]/div/p/img')
        src = img.get_attribute('src')
        title = driver.find_element_by_xpath('//*[@id="con"]/div[2]/div[1]/h4').text
        bot.sendMessage(chat_id='@yyfactory', text ='BNK event -' + title)
        try:
            bot.sendPhoto(chat_id='@yyfactory', photo =src, timeout=20) 
            time.sleep(2)
        except Exception as e : pass
        bot.sendMessage(chat_id='@yyfactory', text = BNKurl)    
        
    else: None   

#'타행이벤트' 라는 폴더를 생성
txt_dir=os.getcwd()+'/타행이벤트/'
if(os.path.isdir(txt_dir)):
    None
else:
    os.mkdir(txt_dir)

    
def txt_mk(name,bank):          #각 은행의 이벤트 명을 저장할 txt파일생성
    if(os.path.isfile(txt_dir+name)):
        None
    else:
        with open(txt_dir+name,'w') as f:
            sel=eval('%s_sel'%bank)
            f.write(eval('%s_event_road(soup,sel)'%bank))
            f.close()

def compare_tit(name,bank):     #각은행의 이벤트명 비교해주는 함수 
    with open(txt_dir+name,'r') as f:
        before = f.readline()
        sel=eval('%s_sel'%bank)
        title = eval('%s_event_road(soup,sel)'%bank)
        if(before != title):
            with open(txt_dir+name,'w') as f:
                sel=eval('%s_sel'%bank)
                f.write(eval('%s_event_road(soup,sel)'%bank))
                f.close()
            eval('%s_png()'%bank)    
        else:
            print('같다')
       

    
    
#위의 함수를 이용하여 각은행의 이벤트명, 이벤트가 올라온날짜를 확인하여 txt에 새로운 이벤트를 저장하고 텔레그램으로 메세지를 전달하는 과정
name1='ibk.txt'
bank1='IBK'
soup = Html_print(IBKurl)
txt_mk(name1,bank1)
    
name2='KB.txt'
bank2='KB'
soup = Html_print(KBurl)
txt_mk(name2,bank2)

name3='NH.txt'
bank3='NH'
soup = Html_print(NHurl)
txt_mk(name3,bank3)

name4='WOORI.txt'
bank4='WOORI'
soup = Html_print(WOORIurl)
txt_mk(name4,bank4)

name5='HANA.txt'
bank5='HANA'
soup = Html_print(HANAurl)
txt_mk(name5,bank5)

name6='BNK.txt'
bank6='BNK'
soup = Html_print(BNKurl)
txt_mk(name6,bank6)

soup = Html_print(IBKurl)
compare_tit(name1,bank1)
    
soup = Html_print(KBurl)
compare_tit(name2,bank2)
    
soup = Html_print(NHurl)
compare_tit(name3,bank3)
   
soup = Html_print(WOORIurl)
compare_tit(name4,bank4)
   
soup = Html_print(HANAurl)
compare_tit(name5,bank5)
    
soup = Html_print(BNKurl)
compare_tit(name6,bank6)


driver.close()


C:\Users\LG\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: DeprecationWarning: use options instead of chrome_options


같다
같다
같다
같다
같다
같다


# 제작이유 : 마케터라면 타행의 이벤트의 민감해야된다고 생각이 들었었고, 내가 각  은행의 이벤트를 찾아볼려고 사이트에 접속하는것이 귀찮을수도 있기때문에 만들었고, 데이터가 곧 자산이라는 생각이 있기때문에 타행의 이벤트들의 데이터를 보고 추후의 새로운 무엇을 해 볼수도 있다는 생각에 만들게 되었습니다.